# UNIFICACION TABLA FINAL

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as snf
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score,mean_squared_error


In [2]:
Censo = pd.read_csv("Censo.csv")
Emisiones = pd.read_csv("Emisiones.csv")
Hospitalizaciones= pd.read_csv("Hospitalizaciones.csv", encoding="iso-8859-1")

In [3]:
Hospitalizaciones.head(3)

,Diagnóstico_ID,Año,Diagnóstico,Sexo,Provincia,Provincia_ID,Hospitalizaciones
0,1,2012,"Neoplasia maligna de tráquea, bronquios y pulmón",Hombres,A Coruña,15,9320
1,1,2012,"Neoplasia maligna de tráquea, bronquios y pulmón",Hombres,Albacete,2,1522
2,1,2012,"Neoplasia maligna de tráquea, bronquios y pulmón",Hombres,Alicante,3,9855


In [4]:
Tabla_Final = Hospitalizaciones.merge(
    Emisiones,
    on = ['Provincia_ID','Provincia','Año'],
    how = 'left'
).merge(
    Censo,
     on = ['Provincia_ID','Provincia','Año','Sexo'],
    how = 'left'
)

In [5]:
Tabla_Final['Porcentaje_H'] = 0.0  # Inicializar la columna

# Recorrer el DataFrame fila por fila
for index, row in Tabla_Final.iterrows():
    # Obtener los valores de Diagnóstico, Provincia, Sexo, Hospitalizaciones y Habitantes
    diagnostico = row['Diagnóstico']
    provincia = row['Provincia']
    sexo = row['Sexo']
    hospitalizaciones = row['Hospitalizaciones']
    habitantes = row['Habitantes']
    
    # Calcular el porcentaje de hospitalizaciones por habitantes (evitando división por cero)
    if habitantes > 0:
        porcentaje = round ((hospitalizaciones / habitantes) * 100,2)
    else:
        porcentaje = 0.0
    
    # Asignar el valor calculado a la nueva columna 'Porcentaje_H'
    Tabla_Final.at[index, 'Porcentaje_H'] = porcentaje


## Creacion de Columnas Metales Pesados e Indice Contaminacion para modelo predictivo

In [6]:
Tabla_Final['Metales Pesados'] = Tabla_Final [['As','Cd','Ni','Pb']].sum(axis=1).round(4)

In [7]:
Tabla_Final['Indice_Contaminación']= (0.5*Tabla_Final['PM25']+ 0.3*Tabla_Final['PM10']+ 0.2*Tabla_Final['BaP']).round(4)

In [8]:
Tabla_Final = Tabla_Final[~Tabla_Final['Provincia'].isin(['Ceuta', 'Melilla'])]

In [9]:
Tabla_Final['Habitantes'] = Tabla_Final['Habitantes'].astype(int)

In [10]:
Tabla_Final.dtypes

Diagnóstico_ID            int64
Año                       int64
Diagnóstico              object
Sexo                     object
Provincia                object
Provincia_ID              int64
Hospitalizaciones         int64
As                      float64
BaP                     float64
Cd                      float64
Ni                      float64
PM10                    float64
PM25                    float64
Pb                      float64
Habitantes                int64
Porcentaje_H            float64
Metales Pesados         float64
Indice_Contaminación    float64
dtype: object

In [11]:
Tabla_Final.to_csv('Tabla_Final.csv', index=False , encoding = 'utf-8')

# MODELO PREDICTIVO

## Random Forest

In [12]:
import joblib

In [13]:
Tabla_Final[(Tabla_Final['Diagnóstico_ID'] == 7) & (Tabla_Final['Provincia_ID'] == 5)& (Tabla_Final['Año'] == 2012)] 

,Diagnóstico_ID,Año,Diagnóstico,Sexo,Provincia,Provincia_ID,Hospitalizaciones,As,BaP,Cd,Ni,PM10,PM25,Pb,Habitantes,Porcentaje_H,Metales Pesados,Indice_Contaminación
675,7,2012,Infecciones/enfermedades de las vías respirato...,Hombres,Ávila,5,380,0.4654,0.1264,0.1242,2.6689,19.2809,12.0044,0.0046,86617,0.44,3.2631,11.8118
727,7,2012,Infecciones/enfermedades de las vías respirato...,Mujeres,Ávila,5,243,0.4654,0.1264,0.1242,2.6689,19.2809,12.0044,0.0046,84748,0.29,3.2631,11.8118


In [14]:
Tabla_Final = Tabla_Final.drop(columns=['Año','Provincia_ID','Diagnóstico_ID','As','BaP','Cd','Ni','PM10','PM25','Pb'])

In [15]:
Tabla_Final.head(3)

,Diagnóstico,Sexo,Provincia,Hospitalizaciones,Habitantes,Porcentaje_H,Metales Pesados,Indice_Contaminación
0,"Neoplasia maligna de tráquea, bronquios y pulmón",Hombres,A Coruña,9320,549265,1.70,3.2631,12.4681
1,"Neoplasia maligna de tráquea, bronquios y pulmón",Hombres,Albacete,1522,201085,0.76,3.2631,11.8118
2,"Neoplasia maligna de tráquea, bronquios y pulmón",Hombres,Alicante,9855,924452,1.07,3.5353,13.3098


In [16]:
Tabla_Final = pd.get_dummies(Tabla_Final,columns=['Diagnóstico','Provincia','Sexo'])

In [17]:
Tabla_Final.head(3)

,Hospitalizaciones,Habitantes,Porcentaje_H,Metales Pesados,Indice_Contaminación,Diagnóstico_Asma,Diagnóstico_COVID-19,Diagnóstico_EPOC y bronquitis,Diagnóstico_Enfermedad crónica de amígdalas,Diagnóstico_Enfermedades de la circulación pulmonar,...,Provincia_Teruel,Provincia_Toledo,Provincia_Valencia,Provincia_Valladolid,Provincia_Zamora,Provincia_Zaragoza,Provincia_Álava,Provincia_Ávila,Sexo_Hombres,Sexo_Mujeres
0,9320,549265,1.70,3.2631,12.4681,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1522,201085,0.76,3.2631,11.8118,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
2,9855,924452,1.07,3.5353,13.3098,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [18]:
X = Tabla_Final.drop(columns=['Habitantes','Porcentaje_H','Hospitalizaciones'])
y = Tabla_Final['Hospitalizaciones']

In [19]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [20]:
model = RandomForestRegressor(n_estimators=300, max_depth=50, min_samples_split=5, min_samples_leaf=4, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(max_depth=50, min_samples_leaf=4, min_samples_split=5,
                      n_estimators=300, random_state=42)

model.fit(X_train,y_train)

In [21]:
y_pred = model.predict(X_test)

In [22]:
r2 = r2_score(y_test,y_pred)
rmse = np.sqrt(mean_squared_error(y_test,y_pred))

print(f'R2:{r2:.2f}')
print(f'RMSE: {rmse:.2f}')

R2:0.68
RMSE: 6831.16


# Visualizacion

In [35]:
import joblib
import os

# Obtener la ruta actual
ruta_actual = os.getcwd()

# Guardar el modelo en la carpeta actual
joblib.dump(model, os.path.join(ruta_actual, 'modelo_random_forest.pkl'))
joblib.dump(label_encoders, os.path.join(ruta_actual, 'label_encoders.pkl'))

print("Modelo y codificadores guardados exitosamente en:", ruta_actual)

Modelo y codificadores guardados exitosamente en: /Users/josemiguel/Desktop/Proyecto/Analisis
